In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors

from numpy import array
from numpy import max
import numpy as np
import pandas as pd
import math
import folium

In [2]:
from download import download

# We choose daily data
url = "https://opendata.arcgis.com/datasets/2ab16a5fb61f42c1a689fd9cc466383f_0.csv"
path_target = "datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv"
download(url, path_target, replace=False)


Replace is False and data exists, so doing nothing. Use replace==True to re-download the data.


'datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv'

In [3]:
occ_df = pd.read_csv(path_target)

Que veut-on représenter? 
   +  Toutes les villes de notre data set en Occitanie
   +  La question est quand ?
   + Peut on faire varier la date interactivement
   + Voila pourquoi on veut une video, on va voire la pollution évoluer
   
J'ai reflechi cette nuit et on developpera ici:
   + Une map de l'occitanie qui affiche les données journalières
   + Elle sera interactive et on pourra choisir le jour qu'on veut
   + Il faudra certainement et au préalable nettoyer nos données

In [4]:
# On va traiter les données afin de rajouter une colonne de la date

occ_df['jour'] = pd.to_datetime(occ_df['date_debut']).dt.to_period('D')

C:\Users\megan\Anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1269: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


In [5]:
occ_df.nom_station.unique()

array(['Peyrusse Vieille Rural', 'Montpellier - Pompignane Trafic',
       'Mende - Vernede urbain', 'Perpignan Sud - urbain', 'Albi Urbain',
       'Toulouse-Jacquier Urbain', 'Agathois-Piscénois - Périurbain',
       'Saze - rural', 'Toulouse-Berthelot Urbain',
       'Montpellier Nord - Périurbain', 'Toulouse-Mazades Urbain',
       'Montpellier - Prés d Arènes Urbain', 'Lourdes-Lapaca Urbain',
       'Castres-Travet Urbain', 'Montpellier - Chaptal Urbain',
       'Tarbes-Dupuy Urbain', 'Nîmes - Planas - Trafic',
       'La Calmette - Périurbain', 'Lunel-Viel - Périurbain',
       'Saint Gaudens-Miramont Industriel',
       'Saint Gaudens-Daurat Industriel', 'Nîmes Sud - urbain',
       'Montgiscard Urbain', 'Belesta en Lauragais Rural',
       'Montpellier Sud - Périurbain', 'Perpignan Centre - urbain',
       'Saint Estève - périurbain', 'Montpellier - Saint Denis Trafic',
       'Blagnac Aéroport Piste ', 'Toulouse-Périphérique Trafic',
       'Blagnac Aéroport Trafic ', 'Toulous

In [6]:
occ_df.jour.unique() # 362 jours, on fera varier notre map selon ces jours.

<PeriodArray>
['2019-12-27', '2020-02-26', '2019-05-27', '2019-12-28', '2020-02-19',
 '2019-08-06', '2019-12-29', '2020-02-20', '2019-12-30', '2019-05-14',
 ...
 '2020-03-23', '2020-03-24', '2019-12-26', '2020-03-25', '2020-03-26',
 '2020-03-27', '2020-02-18', '2020-02-13', '2020-02-14', '2019-06-19']
Length: 362, dtype: period[D]

On va prendre le tableau d'un jour en particulier:

In [7]:
occ_day1 = occ_df[occ_df['jour'] == '2020-03-26']
occ_day1.nom_station.unique()

array(['Peyrusse Vieille Rural', 'Tarbes-Dupuy Urbain',
       'Toulouse-Périphérique Trafic', 'Toulouse Port de l Embouchure',
       'Agathois-Piscénois - Périurbain', 'Toulouse-Mazades Urbain',
       'Saint-Gaudens urbain', 'Toulouse-Jacquier Urbain',
       'Lunel-Viel - Périurbain', 'Perpignan Centre - urbain',
       'Lourdes-Paradis Trafic', 'Castres-Travet Urbain',
       'Perpignan Sud - urbain', 'Lourdes-Lapaca Urbain', 'Saze - rural',
       'Albi Urbain', 'Montpellier - Pompignane Trafic',
       'Nîmes Sud - urbain', 'Toulouse - Rte Albi Trafic',
       'Montgiscard Urbain', 'Belesta en Lauragais Rural',
       'Toulouse-Berthelot Urbain', 'Blagnac Aéroport Piste ',
       'Blagnac Aéroport Trafic ', 'Montpellier - Chaptal Urbain',
       'Montpellier - Prés d Arènes Urbain',
       'Saint Gaudens-Daurat Industriel', 'Saint Estève - périurbain',
       'Nîmes - Planas - Trafic', 'Montpellier Sud - Périurbain',
       'Biterrois-Narbonnais - Périurbain',
       'Saint Gaud

In [8]:
# Ce qui nous interesse désormais dans ce tableau
# nom de la station et de la ville
# lattitude et longitude
# valeur Ozone

occ_day1 = occ_day1[occ_day1['nom_poll'] == 'O3']
occ_day1 = occ_day1[['X','Y','nom_com', 'nom_station', 'valeur', 'jour']]

In [9]:
(occ_day1[['valeur']] - np.mean(occ_day1[['valeur']]))/ np.std(occ_day1[['valeur']])

valeur
50799  0.962238
50802 -0.789730
50803  0.434071
50806 -0.184270
50807  0.369661
50817  0.253722
50829 -0.351738
50830  0.318132
50832 -1.974884
50858  0.150665
50859  0.318132
50861  0.910710
50863 -0.931433
50864  1.580580
50866  0.704596
50870 -0.325974
50871 -0.815494
50881 -1.034490
50905  1.206998
50906  1.116823
50914  0.627303
50918  0.124901
50921 -2.670519

In [10]:
occ_day1['logvaleur'] = np.exp(occ_day1[['valeur']])/10e35
occ_day1['centrer_reduire'] = (occ_day1[['valeur']] - np.mean(occ_day1[['valeur']]))/ np.std(occ_day1[['valeur']])

In [11]:
occ_day1 # nombre très raisonnable de données

X        Y                nom_com  \
50799  4.679690  43.9429                   SAZE   
50802  1.438610  43.6236               TOULOUSE   
50803  4.374220  43.8344                  NIMES   
50806  1.571110  43.4575            MONTGISCARD   
50807  1.821940  43.4414   BELESTA-EN-LAURAGAIS   
50817  0.179722  43.6303       PEYRUSSE-VIEILLE   
50829  1.418610  43.5756               TOULOUSE   
50830  2.839870  42.7198           SAINT-ESTEVE   
50832  2.230000  43.6106                CASTRES   
50858  3.911560  43.5706                 LATTES   
50859  3.188810  43.4068             CORNEILHAN   
50861  3.800820  43.6954     SAINT-GELY-DU-FESC   
50863  4.265810  43.9254            LA CALMETTE   
50864  3.886890  43.5915            MONTPELLIER   
50866 -0.038889  43.0969                LOURDES   
50870  0.744722  43.0944  MIRAMONT-DE-COMMINGES   
50871  1.443890  43.5872               TOULOUSE   
50881  1.385410  43.5970               TOULOUSE   
50905  0.062222  43.2303                 TARBES   
50906  2.570700  44.3570                  RODEZ   
50914  3.504830  43.2878                   AGDE   
50918  2.524840  43.3233       CAUNES-MINERVOIS   
50921  2.899700  42.6958              PERPIGNAN   

                              nom_station  valeur        jour     logvaleur  \
50799                        Saze - rural    79.0  2020-03-26  2.038281e-02   
50802             Toulouse-Mazades Urbain    65.4  2020-03-26  2.528478e-08   
50803                  Nîmes Sud - urbain    74.9  2020-03-26  3.377977e-04   
50806                  Montgiscard Urbain    70.1  2020-03-26  2.779990e-06   
50807          Belesta en Lauragais Rural    74.4  2020-03-26  2.048847e-04   
50817              Peyrusse Vieille Rural    73.5  2020-03-26  8.329989e-05   
50829            Toulouse-Jacquier Urbain    68.8  2020-03-26  7.576356e-07   
50830           Saint Estève - périurbain    74.0  2020-03-26  1.373383e-04   
50832               Castres-Travet Urbain    56.2  2020-03-26  2.554759e-12   
50858        Montpellier Sud - Périurbain    72.7  2020-03-26  3.742905e-05   
50859   Biterrois-Narbonnais - Périurbain    74.0  2020-03-26  1.373383e-04   
50861       Montpellier Nord - Périurbain    78.6  2020-03-26  1.366301e-02   
50863            La Calmette - Périurbain    64.3  2020-03-26  8.416571e-09   
50864  Montpellier - Prés d Arènes Urbain    83.8  2020-03-26  2.476724e+00   
50866               Lourdes-Lapaca Urbain    77.0  2020-03-26  2.758513e-03   
50870   Saint Gaudens-Miramont Industriel    69.0  2020-03-26  9.253782e-07   
50871           Toulouse-Berthelot Urbain    65.2  2020-03-26  2.070142e-08   
50881                 Station Pedagogique    63.5  2020-03-26  3.781809e-09   
50905                 Tarbes-Dupuy Urbain    80.9  2020-03-26  1.362773e-01   
50906                        Rodez Urbain    80.2  2020-03-26  6.767331e-02   
50914     Agathois-Piscénois - Périurbain    76.4  2020-03-26  1.513904e-03   
50918            Caunes-Minervois - rural    72.5  2020-03-26  3.064432e-05   
50921           Perpignan Centre - urbain    50.8  2020-03-26  1.153877e-14   

       centrer_reduire  
50799         0.962238  
50802        -0.789730  
50803         0.434071  
50806        -0.184270  
50807         0.369661  
50817         0.253722  
50829        -0.351738  
50830         0.318132  
50832        -1.974884  
50858         0.150665  
50859         0.318132  
50861         0.910710  
50863        -0.931433  
50864         1.580580  
50866         0.704596  
50870        -0.325974  
50871        -0.815494  
50881        -1.034490  
50905         1.206998  
50906         1.116823  
50914         0.627303  
50918         0.124901  
50921        -2.670519

In [12]:
import branca.colormap as cm

linear = cm.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=-3, vmax=1
)

In [26]:
map = folium.Map(location=[43.9333, 2.15], 
                 zoom_start=7.5, 
                 tiles='Stamen Terrain')

for i in range(0,len(occ_day1)):
   folium.Circle(
      location=[occ_day1.iloc[i]['Y'], occ_day1.iloc[i]['X']],
      popup=occ_day1.iloc[i]['nom_station'],
      radius=occ_day1.iloc[i]['valeur']*100,
      color='black',
      fill=True,
      fill_color=linear(occ_day1.iloc[i]['centrer_reduire']),
       fill_opacity=0.5,
       opacity=0.4,
   ).add_to(map)

map


# TROUVER UNE BONNE ECHELLE POUR BIEN VOIR LES DIFFERENCES
# selectionner que certaines stations??


In [ ]:
import matplotlib.animation as animation
# Faire un truc interactif où on selectionne le mois et le jour
# Comment obtient on une image de ce truc ?
# Animons tout ça...